# 5. Régression linéaire sur un jeu de données synthétique

## Jeu d'entraînement

Nous considèrons un jeu d'entraînement constitué de $N$ points générés par un processus que nous contrôlons (par ex. $f(x)=2x$, ou $f(x)=e^x cos(2\pi sin(\pi x))$) et sujet à un bruit que nous contrôlons également.

Nous commençons par importer les différentes librairies que nous allons utiliser. Nous fixons également la taille de la police de caractères pour l'affichage des graphiques.

In [ ]:
import numpy as np
%matplotlib inline
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import Ridge
from math import sqrt
from matplotlib import pyplot as plt
from matplotlib import rcParams

rcParams.update({'font.size': 16})

La constante `N` représente le nombre de points générés pour le jeu d'entraînement. Le vecteur `x` contient les abscisses de chacun de ces `N` points. Il est généré en découpant en `N` intervalles réguliers le segment de nombres réels $[0.05,0.95]$. La constante `bruit` modélise la quantité de bruit ajouté à chaque point. Le vecteur `alea` contient le bruit pour chacun des `N` points. Il est généré par échantillonnage d'une __[loi de normale](https://en.wikipedia.org/wiki/Normal_distribution#Definition)__ de moyenne nulle et d'écart-type `bruit`. Cet échantillonnage est réalisé par la fonction __[numpy.random.randn](https://docs.scipy.org/doc/numpy-1.15.1/reference/generated/numpy.random.randn.html)__.

In [ ]:
N=20
x=np.linspace(0.05,0.95,N)
bruit=0.8
alea = bruit * np.random.randn(N)

In [ ]:
x

In [ ]:
alea

Nous générons le vecteur d'ordonnées `y` associé aux abscisses `x` soit par un processus linéaire bruité de forme $f(x)=2x$, soit par un processus non-linéaire bruité de forme $f(x)=2x$, ou $f(x)=e^x cos(2\pi sin(\pi x))$.

In [ ]:
def processus_lineaire(x):
    return 2*x

processus = processus_lineaire

y = processus(x) + alea

In [ ]:
y

Nous définissons des fonctions pour afficher les données du jeu d'entraînement.

In [ ]:
def plot_train_data():
    plt.plot(x, y, "o", markersize=10, alpha=0.3, label='Entraînement')

plot_title = "N=%i, bruit=%.2f (entraînement)"%(N,bruit)
    
def plot_legend():
    plt.legend(loc='best')
    plt.ylim([-7,7])
    plt.xlabel("x")
    plt.ylabel("y")
    plt.title(plot_title)
    plt.tight_layout()
    plt.grid()
    plt.show()
    
def plot_init():
    plt.figure(figsize=(8, 6))

def plot_process(max_x):
    xplot = np.linspace(0,max_x, 200)
    plt.plot(xplot, processus(xplot), lw=2, label='Processus inconnu')

def plot_model(clf, model_name, max_x):
    xplot = np.linspace(0,max_x, 200)
    plt.plot(xplot, clf.predict(xplot.reshape(-1, 1)), lw=2, label=model_name)

plot_init()
plot_train_data()
plot_legend()

## Modèle linéaire

Nous apprenons un modèle linéaire à partir des données d'entraînement. La méthode `fit` de l'objet [`sklearn.linear_model.LinearRegression`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html) attend en premier paramètre une matrice de données dont chaque ligne est un individu et chaque colonne est une variable. Or, nos observation `x` sont sous la forme d'un vecteur et non d'une matrice (car ici chaque individu n'est décrit que par une seule variable, son abscisse). Ainsi, nous transformons le vecteur `x` en une matrice de `N` lignes et `1` colonne grâce à la notation [`x.reshape(-1, 1)`](https://docs.scipy.org/doc/numpy/reference/constants.html#numpy.newaxis) (nous pouvons aussi utiliser [`x[:, np.newaxis]`](https://docs.scipy.org/doc/numpy/reference/constants.html#numpy.newaxis).

Entrainez un modèle de régression linéaire sur ce dataset :

Affichez les données d'entrainement, notre processus et la régression linéaire obtenue par apprentissage sur le même graphique. Vous disposez des fonctions
* `plot_process(max_x)` : `max_x` correspond à l'abscisse maxium de notre graphique (ici la valeur `1.2` permet mieux visualiser nos donnéées)
* `plot_model(clf, model_name, max_x)`: `clf` correspond à l'instance de régression linéaire, `model_name` correspond au nom à afficher dans la légende

## Modèle polynomial

Nous entrainons maintenant des modèles polynomiaux. Nous utilisons pour cela la classe [`sklearn.preprocessing.PolynomialFeatures`](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.PolynomialFeatures.html).

Entrainez et afficher des modèles polynomiaux pour notre dataset avec différents degrés de polynôme (3, 6 et 12 par exemple). Pour simplifier pouvez créer un [`Pipeline scikit-learn`](https://scikit-learn.org/stable/modules/compose.html#pipeline) avec la fonction [`sklearn.pipeline.make_pipeline`](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.make_pipeline.html). Un Pipeline est une succession d'opérations scikit-learn disposant toute d'une méthode `fit` et `predict`, vous pouvez le manipuler comme un estimateur classique scikit-learn.

## Ridge

Nous apprenons ensuite un modèle polynomiale régularisé par la méthode _ridge_ avec différents choix pour le coefficient de régularisation.

Entrainez et afficher des modèles polynomiaux de degré 12 régularisés pour notre dataset avec différents coefficients de régularisation (0.5, 0.1 et 0.01 par exemple). Ici aussi, vous pouvez utiliser la fonction [`sklearn.pipeline.make_pipeline`](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.make_pipeline.html).

## Jeu de test

Nous comparons les modèles sur un jeu de test en affichant la métrique `rmse` (root mean squared error).

La cellule suivante créée un jeu de test avec du bruit à partir du processus défini au début du notebook.

Appliquez différents modèles sur ce jeu de test, calculez le RMSE ([sklearn.metrics.mean_squared_error](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_squared_error.html) avec `sqrt`), affichez vos regresseurs et les données de test (à l'aide de la fonction `plot_test_data`). Quel est le meilleur modèle ? Quel est l'impact de la régularisation sur une régression polynomiale de degré élevé ?

In [ ]:
N_test = 20
max_x = 1.2
bruit_test = bruit
x_test=max_x * np.random.random(N_test)
alea_test = bruit_test*np.random.randn(N_test)
y_test = processus(x_test) + alea_test

plot_title = "N=%i, bruit=%.2f (test)"%(N_test,bruit_test)

def plot_test_data():
    plt.plot(x_test, y_test, "o", markersize=10, alpha=0.3, label='Test')

## Pistes d'explorations

Pour $N=10$, $f(x)=2x$ et un bruit nul, comparer sur le jeu d'entraînement les graphes de modèles de complexité croissante. Quel est le modèle qui s'adapte le mieux aux données ? Quel est le modèle qui fait les meilleurs prédictions ?

Répéter ces expériences en modifiant les conditions, par exemple : ($N=10$, $f(x)=2x$, $bruit=1$) ou ($N=100$, $f(x)=2x$, $bruit=1$), etc.

Décrire la relation entre la complexité du modèle (son nombre de paramètres), sa capacité à s'adapter aux données d'entraînement et sa capacité à prédire sur de nouvelles données.

Répéter les expériences avec un processus génératif non linéaire, par exemple : $f(x)=e^x cos(2\pi sin(\pi x))$. Vous pouvez utiliser la fonction suivante pour remplacer le processus linéaire :

In [ ]:
def processus_non_lineaire(x):
    return np.exp(x) * np.cos(2*np.pi*np.sin(np.pi*x))

Explorer la relation entre la complexité d'un modèle et les valeurs apprises pour ses paramètres.

Exemple de code pour accéder aux valeurs des coefficient d'un modèle linéaire entraîné :

In [ ]:
poly_model[2].steps[1][1].coef_

In [ ]:
ridge_model[2].steps[1][1].coef_